In [14]:
%pip install -e "./Pitmodule"
from Pitmodule.pitframe import *
from Pitmodule.population import PopulationFrame
import pandas as pd
import numpy as np
populacje2020 = PopulationFrame()
lud_gminy = populacje2020.data["Gminy"]
pit2020 = PitFrame()

Obtaining file:///C:/Users/adomoslaws001/nypd-finalassignment/Pitmodule
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: Pitmodule
    Found existing installation: Pitmodule 1.0
    Uninstalling Pitmodule-1.0:
      Successfully uninstalled Pitmodule-1.0
  Running setup.py develop for Pitmodule
Note: you may need to restart the kernel to use updated packages.


In [36]:
pit2020 = PitFrame(year=2020)

pit_gminy = pit2020.data["Gminy"]
pit_gminy.reset_index(inplace = True)

pit_gminy['dochod'] = pit_gminy["naleznosci"].apply(lambda x:  x/0.17)
pit_gminy = pit_gminy[["jst","gt", "dochod"]]

lud_gminy = populacje2020.data["Gminy"][["gt","jst", "ludnosc"]]
pit_gminy= pit_gminy.astype({'gt':int})


dochody_gminy = pd.merge(pit_gminy, lud_gminy,how="inner", on=['gt','jst'])
dochody_gminy['dochod-per-osoba'] = dochody_gminy["dochod"]/dochody_gminy["ludnosc"]

dochody_gminy

AttributeError: 'PitFrame' object has no attribute 'dtypes'

In [15]:
pit2020.count_tax_income()
# pit2020.get_TASK2(populacje2020)
pit2020.get_TASK2(populacje2020)
pit2020.data['Gminy']

AssertionError: 